In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
# Pongo si estoy juntando data de train o de test
archivo_test = 0

if archivo_test == 0: 
    s3_url = f"viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Train"
elif archivo_test == 1:
    s3_url = f"viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Test"

## Armo Train y Validacion

In [3]:
if archivo_test == 0: 
    s3_url = f"viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Train"
    
    df1 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202301.parquet")
    df2 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202302.parquet")
    df3 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202303.parquet")
    df4 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202304.parquet")
    df5 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202305.parquet")
    df6 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202306.parquet")
    df7 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202307.parquet")
    df8 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202308.parquet")
    #df12 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202212.parquet")
    #df11 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202211.parquet")
    

In [4]:
## unir las meses
if archivo_test == 0:
    dfs = [df1, df2, df3, df4, df5, df6, df7, df8] #df11, df12

    df = pd.concat(dfs, ignore_index=True)

In [5]:
if archivo_test == 0:
    from sklearn.model_selection import train_test_split

    X_train, X_val, y_train, y_val = train_test_split(
        df.loc[:, df.columns != 'target_fraudes'], df['target_fraudes'], test_size=0.3, random_state=42
    )

In [6]:
if archivo_test == 0:
    #Pongo el nombre de las columnas y vuelvo a juntar con el target
    feature_names = df.loc[:, df.columns != 'target_fraudes'].columns

    trainX = pd.DataFrame(X_train, columns=feature_names)
    trainX["target"] = y_train

    valX = pd.DataFrame(X_val, columns=feature_names)
    valX["target"] = y_val

In [7]:
if archivo_test == 0:
    s3_url = f"s3://viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Data4Model/TargetTodos/Train/Train.parquet"
    trainX.to_parquet(path=s3_url, engine='pyarrow', compression='gzip')

    s3_url = f"s3://viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Data4Model/TargetTodos/Validation/Validation.parquet"
    valX.to_parquet(path=s3_url, engine='pyarrow', compression='gzip')
    print('termine de salvar el parquet del train y val')

termine de salvar el parquet del train y val


## Armo archivos para test del modelo

In [8]:
if archivo_test == 1: 
    s3_url = f"viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Test"
    
    #dfT1 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202308.parquet")
    dfT2 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202309.parquet")
    dfT3 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202310.parquet")
    dfT4 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202311.parquet")
    dfT5 = pd.read_parquet(f"s3://{s3_url}/Preproc_Fr_202312.parquet")
    

In [9]:
## unir las meses
if archivo_test == 1:
    dfTs = [ dfT2, dfT3, dfT4, dfT5] #dfT1,

    dfT = pd.concat(dfTs, ignore_index=True)
    dfT = dfT.rename(columns={"target_fraudes": "target"})


    s3_url = f"s3://viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Data4Model/TargetTodos/Test/Test.parquet"
    dfT.to_parquet(path=s3_url, engine='pyarrow', compression='gzip')
    print('termine de salvar el parquet del test')